#### 기본적인 DL 모델링

In [16]:
import sys
import pandas as pd
import tensorflow as tf
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

import matplotlib.pyplot as plt
from freeman.plt_setting import plt_settings
from freeman.evaluation import regression_evaluation, f_importances, plot_actual_pred

sys.path.append('..')
from utils.data_manager import read_data

# 한글처리 지원
plt_settings()

#### 데이터 불러오기

In [2]:
df_data = read_data('2nd pp counts-base-on-cons-1st')

feature_columns = [
    'year', 'month', 'day', 'dayofweek', 'dayofyear',
    '사번코드숫자', '사번숫자', '사업소코드', '계약전력', 
    'line_cnts', 'pole_cnts', 'sl_cnts'
]
target_column = '총공사비'

In [3]:
df_X = df_data[feature_columns + [target_column]]
df_y = df_X.pop(target_column)

X_train, X_test, y_train, y_test = \
    train_test_split(df_X, df_y, test_size=0.1)

#### 데이터 스케일링

In [5]:
standard_scaler = StandardScaler()
X_train_scaled = standard_scaler.fit_transform(X_train)
X_test_scaled = standard_scaler.transform(X_test)

#### DL Modeling

In [7]:
model_mlp = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1:])),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])
model_mlp.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mae']
)
model_mlp.summary()

2023-09-16 17:40:20.212311: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-16 17:40:20.254413: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-16 17:40:20.254705: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1664      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 12033 (47.00 KB)
Trainable params: 12033 (47.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
model_mlp.fit(
    X_train_scaled, y_train, 
    epochs=200, verbose=2, validation_split=0.2
)

Epoch 1/200


341/341 - 1s - loss: 13769787834368.0000 - mae: 1811879.1250 - val_loss: 13977730940928.0000 - val_mae: 1899037.7500 - 715ms/epoch - 2ms/step
Epoch 2/200
341/341 - 1s - loss: 13771199217664.0000 - mae: 1804267.7500 - val_loss: 13974881959936.0000 - val_mae: 1883938.2500 - 679ms/epoch - 2ms/step
Epoch 3/200
341/341 - 1s - loss: 13768436219904.0000 - mae: 1798858.8750 - val_loss: 13971738329088.0000 - val_mae: 1907028.6250 - 691ms/epoch - 2ms/step
Epoch 4/200
341/341 - 1s - loss: 13765906006016.0000 - mae: 1802323.6250 - val_loss: 13968059924480.0000 - val_mae: 1902632.2500 - 659ms/epoch - 2ms/step
Epoch 5/200
341/341 - 1s - loss: 13767946534912.0000 - mae: 1800536.2500 - val_loss: 13979713798144.0000 - val_mae: 1913485.3750 - 669ms/epoch - 2ms/step
Epoch 6/200
341/341 - 1s - loss: 13762949021696.0000 - mae: 1806641.7500 - val_loss: 13974738305024.0000 - val_mae: 1881413.3750 - 669ms/epoch - 2ms/step
Epoch 7/200
341/341 - 1s - loss: 13760493256704.0000 - mae: 1802349.3750 - val_loss: 139

In [13]:
loss = model_mlp.evaluate(X_test_scaled, y_test, verbose=0)
loss

[11605472444416.0, 1681804.375]

In [14]:
pred_mlp = model_mlp.predict(X_test_scaled, verbose=0)

In [15]:
_ = regression_evaluation(y_test, pred_mlp)

R2_SCORE: 0.582032, MAPE: 29.432545, MSE: 11605471856705.865234, RMSE: 3406680.474701, MAE: 1681804.058785


#### BayesianOptimization MLP

In [21]:
def create_mlp_model(units1, units2, units3):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units1, activation='relu', input_shape=(X_train.shape[1:])),
        tf.keras.layers.Dense(units2, activation='relu'),
        tf.keras.layers.Dense(units3, activation='relu'),
        # 회귀모델에서는 출력층에 활성화함수를 사용하지 않음
        tf.keras.layers.Dense(1)
    ])
    model.compile(
        optimizer='adam', 
        loss='mean_squared_error', 
        metrics=['mae', 'mape']
    )
    return model

In [23]:
# 임의 실행
_start = datetime.now()
model_bo_mlp = create_mlp_model(256, 256, 128)
model_bo_mlp.fit(X_train_scaled, y_train, epochs=100, verbose=0)
loss = model_bo_mlp.evaluate(X_test_scaled, y_test, verbose=0)
print(f'Loss: {loss}')
print(f'Processing Time: {datetime.now() - _start}')
# scaling 전에는 16.xx 였는데 13.xx로 떨어짐

Loss: [11244976209920.0, 1637490.125, 28.7490177154541]
Processing Time: 0:01:14.586101


In [24]:
pred_bo_mlp = model_bo_mlp.predict(X_test_scaled, verbose=0)
_ = regression_evaluation(y_test, pred_bo_mlp)

R2_SCORE: 0.595015, MAPE: 28.749016, MSE: 11244975805035.013672, RMSE: 3353352.919845, MAE: 1637489.816711


In [25]:
def target_func_mlp(units1, units2, units3, epochs):
    model = create_mlp_model(int(units1), int(units2), int(units3))
    model.fit(X_train_scaled, y_train, epochs=int(epochs), verbose=0)
    loss = model.evaluate(X_test_scaled, y_test, verbose=0)
    return -loss[2]

In [26]:
pbounds_mlp = {
    'units1': (128, 512),
    'units2': (64, 256),
    'units3': (32, 128),
    'epochs': (50, 200)
}

In [28]:
SEED = 123
optimizer_mlp = BayesianOptimization(
    f=None,
    pbounds=pbounds_mlp,
    verbose=2,
    random_state=SEED
)

utility = UtilityFunction(
    kind='ucb', # 탐색과 활용 사이의 균형 유지, 불확실한 지점을 더 많이 탐색
    kappa=2.5,  # UCB전략에서 얼마의 불확실성을 고려할지 지정
                # 값이 높으면 탐색을, 값이 낮으면 활용을 강조함
    xi=0.0      # 탐색전략에서 사용되는 파라미터로 얼마나 큰 개선을 고려할지 조정
                # 값이 높으면 큰 개선을 값이 낮으면 작은 개선을 탐색함    
)

In [29]:
# 임의실행
next_point = optimizer_mlp.suggest(utility)
target = target_func_mlp(**next_point)
print(f'MAPE: {-target}')

MAPE: 28.67093276977539


In [30]:
optimizer_mlp.register(params=next_point,target=target)

In [31]:
_start = datetime.now()
print('|  Index |  Target | Units1 | Units2 | Units3 | Epochs |     Time     |')
print('|--------+---------+--------+--------+--------+--------+--------------|')

for n_iter in range(20):
    __start = datetime.now()
    next_point = optimizer_mlp.suggest(utility)
    target = target_func_mlp(**next_point)
    optimizer_mlp.register(params=next_point,target=target)
    print(
        f'|{(n_iter+1):>7} |{-target:8.4f} |{int(next_point["units1"]):>7} ' 
        f'|{int(next_point["units2"]):>7} |{int(next_point["units3"]):>7} '
        f'|{int(next_point["epochs"]):>7} |  {str(datetime.now()-__start)[:-3]} |' 
    )
    
print('|--------+---------+--------+--------+--------+--------+--------------|')
print(optimizer_mlp.max)
print(f'Processing Time: {datetime.now() - _start}')

|  Index |  Target | Units1 | Units2 | Units3 | Epochs |     Time     |
|--------+---------+--------+--------+--------+--------+--------------|
|      1 | 30.2167 |    291 |     70 |     73 |     71 |  0:00:53.125 |
|      2 | 29.0478 |    235 |    100 |     94 |    153 |  0:01:53.465 |
|      3 | 27.3826 |    254 |    115 |     61 |    167 |  0:02:02.821 |
|      4 | 28.5088 |    254 |    142 |     32 |    186 |  0:02:16.452 |
|      5 | 29.9241 |    279 |     98 |     53 |    178 |  0:02:10.787 |
|      6 | 31.0467 |    213 |    147 |    102 |     89 |  0:01:05.890 |
|      7 | 29.3200 |    230 |    110 |     76 |    156 |  0:01:54.924 |
|      8 | 28.5836 |    477 |    231 |     83 |     84 |  0:01:02.440 |
|      9 | 28.0370 |    257 |    125 |     60 |    183 |  0:02:14.470 |
|     10 | 29.3948 |    259 |    128 |     70 |    150 |  0:01:50.206 |
|     11 | 27.8041 |    441 |    160 |     72 |    191 |  0:02:19.957 |
|     12 | 28.4907 |    248 |    119 |     50 |    172 |  0:02:0

In [34]:
model_bo_mlp = create_mlp_model(256, 128, 64)
model_bo_mlp.fit(X_train_scaled, y_train, epochs=170, verbose=2)
pred_bo_mlp = model_bo_mlp.predict(X_test_scaled, verbose=0)
_ = regression_evaluation(y_test, pred_bo_mlp)

Epoch 1/170
426/426 - 2s - loss: 55965944643584.0000 - mae: 5224086.0000 - mape: 96.1455 - 2s/epoch - 4ms/step
Epoch 2/170
426/426 - 1s - loss: 23387919351808.0000 - mae: 2682958.2500 - mape: 46.7804 - 781ms/epoch - 2ms/step
Epoch 3/170
426/426 - 1s - loss: 15187058884608.0000 - mae: 2043727.1250 - mape: 36.9128 - 778ms/epoch - 2ms/step
Epoch 4/170
426/426 - 1s - loss: 14768262873088.0000 - mae: 1987633.5000 - mape: 35.6349 - 738ms/epoch - 2ms/step
Epoch 5/170
426/426 - 1s - loss: 14522269040640.0000 - mae: 1947012.7500 - mape: 34.3275 - 771ms/epoch - 2ms/step
Epoch 6/170
426/426 - 1s - loss: 14366454841344.0000 - mae: 1906336.2500 - mape: 33.2617 - 737ms/epoch - 2ms/step
Epoch 7/170
426/426 - 1s - loss: 14256957292544.0000 - mae: 1892350.7500 - mape: 32.6956 - 732ms/epoch - 2ms/step
Epoch 8/170
426/426 - 1s - loss: 14168116690944.0000 - mae: 1874905.0000 - mape: 32.2827 - 753ms/epoch - 2ms/step
Epoch 9/170
426/426 - 1s - loss: 14111848005632.0000 - mae: 1869566.6250 - mape: 31.9809 - 